In [ ]:
# 1. Data Collection and Preprocessing

# Neural responses were recorded from 166 neurons in macaque V1 using a 32-channel array.
# Stimuli were natural images and synthesized textures, presented at 60 ms per image with no blank screens.
# Spike counts were extracted from 40-100 ms after stimulus onset.
# [TODO] Only neurons with at least 15% of their variance attributable to the stimulus were included.

# 2. Feature Extraction from VGG-19

# VGG-19, pre-trained on ImageNet, was used to extract features.
# Conv3_1 was chosen as the best feature layer for V1 response prediction.
# [TODO] Feature maps from VGG-19 were passed through batch normalization before being used for fitting.

# 3. Generalized Linear Model (GLM) Readout

# A GLM with a Poisson loss function was used to map VGG-19 features to neural responses.
# Three regularization terms were applied:
# L1 sparsity: Encourages feature selection.
# Spatial smoothness: Ensures receptive field locality.
# Group sparsity: Encourages pooling from a subset of feature maps.

# 4. Performance Evaluation Using FEV

# Fraction of Explainable Variance Explained (FEV) was computed as:

# 5. Results 
# VGG-19 (Conv3_1) achieved ~51.6% FEV, outperforming LNP (16.3%) and GFB (45.6%).
# The data-driven CNN performed similarly to VGG-19 but required more training data.
# VGG-19's advantage: Achieved high performance with only 20% of the dataset, while the CNN needed the full dataset.

In [ ]:
# Questions

# My analysis gives very poor (- or close to zero) r-squared scores, so:

# 1. Would using 1400 images instead of 7250 explain this poor score?
# 2. Would using a pretrained VGG-19 mode, without Cadena's weights, explain this poor score?
# 3. Would not copying Cadena's image preprocessing (downsampling and cropping) explain this poor score?
# 4. Would not using their regularisation techniques explain this poor score (I don't think so, because they show that it lifts score from ~30% to ~50%)?

# - What size images do they pass into the model? [224x224]
# - They mention that only with at least 15% of their variance attributable to the stimulus were included - is this filtering already applied? [Yes, 166 neurons in data["responses"]]
# - They mention feature maps from VGG-19 were passed through batch normalization before being used for fitting - do I need to do this manually? [I think I need to do this manually]
# - They use feature pooling - do I need to do this?
# - They use feature maps directly, apply spatial pooling on all, then flatten. I was flattening features in batches and not applying spatial pooling. This may turn out to be important for the SimCLR analysis.

In [ ]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
import pickle
import os
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge
import torch.nn.functional as F
from sklearn.metrics import r2_score

DATA_PATH = '../../data/cadena_plosCB19/'
FILE = 'cadena_ploscb_data.pkl'

# ===================================
# Load Cadena's daata
# ===================================
def load_neural_data():
    file_path = os.path.join(DATA_PATH, FILE)
    with open(file_path, "rb") as f:
        data = pickle.load(f)
    return data

data_dict = load_neural_data()

# ===================================
# Clean neural data https://github.com/sacadena/Cadena2019PlosCB/blob/master/cnn_sys_ident/data.py
# ===================================
responses = data_dict['responses'].copy() 
responses[np.isnan(responses)] = 0
data_dict['responses'] = responses

# ===================================
# Extract features from VGG-19
# ===================================

# Load VGG-19 and extract features only up to conv1_1
vgg = models.vgg19(pretrained=True).features[:2].eval()

# Move model to CPU (or GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vgg.to(device)

class ImageDataset(Dataset):
    def __init__(self, images):
        self.images = images
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),  # Resize for VGG-19
            transforms.Grayscale(num_output_channels=3),  # Convert to 3-channel grayscale
            transforms.ToTensor(),  # Convert to PyTorch tensor
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet normalization
        ])

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]

        # Ensure img is in float32 format before converting to PIL
        if isinstance(img, np.ndarray):
            img = img.astype(np.float32)  # Explicitly cast to float32
            img = Image.fromarray(img)  # Convert NumPy to PIL

        return self.transform(img)  # Apply transforms

# Initialize dataset and DataLoader
dataset = ImageDataset(data_dict["images"][:1000])
dataloader = DataLoader(dataset, batch_size=4, shuffle=False, num_workers=0)

# Extract features in batches (to avoid RAM overload)
all_features = []
with torch.no_grad():
    for i, batch in enumerate(dataloader):
        batch = batch.to(device)
        features = vgg(batch)  # Extract Conv1_1 features

        all_features.append(features.cpu())  # Keep shape (batch, C, H, W)

        # Print progress every 100 batches
        if i % 100 == 0:
            print(f"Processed {i * len(batch)} images...")

# Concatenate all extracted features
vgg_features = torch.cat(all_features, dim=0)
print("Final extracted features shape:", vgg_features.shape)  # Expected: (n_images, feature_dim)

# ===================================
# Apply spatial pooling to features, then flatten
# NOTE: Cadena et al. use learned spatial pooling, not fixed pooling like adaptive_avg_pool2d.
# NOTE: Their pooling is encouraged through trainable pooling weights and regularization​.
# TODO: to replicate, might want to use a trainable depthwise convolution, instead of adaptive_avg_pool2d
# TODO: they also apply batch normalisation BEFORE pooling https://github.com/sacadena/Cadena2019PlosCB/blob/master/cnn_sys_ident/vggsysid.py 
# vgg_feats_bn = tf.layers.batch_normalization(vgg_features, training = self.is_training, momentum = 0.9, epsilon = 1e-4, name='vgg_bn', fused =True)
# ===================================

# Apply spatial pooling before flattening
if vgg_features.dim() == 2:  # Likely (n_images, flattened_dim)
    raise ValueError("vgg_features is already flattened! Ensure it retains spatial dimensions.")
pooled_features = F.adaptive_avg_pool2d(vgg_features, (14, 14)) # Reduce spatial size
X_pooled = pooled_features.view(pooled_features.size(0), -1).cpu().numpy() # Flatten pooled maps

X = X_pooled
Y = data_dict["responses"].mean(axis=0)[:1000]

# Features shape: (1000, 12544)
# Responses shape: (1000, 166)
print("Features shape:", X_pooled.shape)
print("Responses shape:", Y.shape)

In [30]:
# ===================================
# Regression
# ===================================

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

ridge = Ridge(alpha=200) # Adjust alpha for regularization strength
ridge.fit(X, Y) # Train on all neurons at once
Y_pred = ridge.predict(X)
r2 = r2_score(Y, Y_pred, multioutput='raw_values')

print("Mean R-squared score:", r2.mean())

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

ridge.fit(X_train, Y_train)  # Train on 80%
Y_pred_test = ridge.predict(X_test)  # Predict on 20%

r2_test = r2_score(Y_test, Y_pred_test, multioutput='raw_values')
print(f"Mean R² score on test set: {np.mean(r2_test):.4f}")  # Should be lower than training R²

# FEV
trial_var = np.var(data_dict["responses"], axis=0)  # Variance across trials
noise_var = np.mean(trial_var, axis=0)  # Average across images
total_var = np.var(Y, axis=0)
mse = mean_squared_error(Y, Y_pred, multioutput='raw_values')
fev = 1 - (mse - noise_var) / (total_var - noise_var)
fev = np.clip(fev, 0, 1)
print(f"Mean FEV across neurons: {np.mean(fev):.4f}")

Mean R-squared score: 0.3201294385227314
Mean R² score on test set: 0.1027
Mean FEV across neurons: 0.2261
